## Account

- properties: [id, name, score]
- methods: getter + setter, toString, compareTo

In [2]:
class Account:
    def __init__(self, id: int, name: str):
        self.id = id
        self.name = name
        self.__score = 0
        
    @property
    def score(self):
        return self.__score
    
    def add_score(self, points: int):
        self.__score += points

    def getLoyaltyPromo(self):
        """15% discount if score is high (>= 100), else no discount"""
        return 0.15 if self.__score >= 100 else 0.0
    def __str__(self):
        return f"Account(id={self.id}, name='{self.name}', score={self.__score})"

## Order

- properties: [id, account, item_list(dict), voucher(str)]
- methods: getVoucherPromo() - 10% getBulkOrderPromo() - 10% setScore(), printBill(), total()

In [3]:
class ItemManager:  # Danh sách món hàng được mua
    def __init__(self) -> None:  # Mặc định khai báo giá trị món hàng là 0
        self.items = {}

    def add_items(self, item) -> None:
        # (item obj, qty)
        if item.id not in self.items:  # sửa __items thành items
            self.items[item.id] = (item, 1)
        else:
            old_qty = self.items[item.id][1]
            self.items[item.id] = (item, old_qty + 1)

    def edit_item(self, id, new_item) -> None:
        if id in self.items:
            self.items[id] = (new_item, self.items[id][1])
        else:
            raise Exception("Item not found")

    def remove_item(self, id) -> None:
        if id in self.items:
            del self.items[id]
        else:
            raise Exception("Item not found")

    def get_price_by_qty(self, id):
        if id in self.items:
            return self.items[id][0].price * self.items[id][1]
        else:
            raise Exception("Item not found")

    def get_sum_qty(self):
        return sum(item[1] for item in self.items.values())

    def get_total_of_list(self):
        return sum(item[0].price * item[1] for item in self.items.values())

    def __str__(self) -> str:
        result = "Danh sách món hàng:\n"
        for item_id, (item, qty) in self.items.items():
            result += f"ID: {item_id}, Tên: {item.name}, Số lượng: {qty}, Đơn giá: {item.price}, Tổng: {item.price * qty}\n"
        result += f"Tổng tiền: {self.get_total_of_list()}"
        return result


In [4]:
class Order:
    def __init__(self, account: Account, order_id: str, item_list: ItemManager, voucher: str = "") -> None:
        self.account = account
        self.order_id = order_id
        self.item_list = item_list
        self.voucher = voucher
        
    def calculate_total(self):
        soluonglon = 0.1 if self.item_list.get_sum_qty() >= 10 else 0 
        khachhangthanthiet = self.account.getLoyaltyPromo()
        voucher = 0.1 if self.voucher != "" else 0
        tong = self.item_list.get_total_of_list() * (1 - soluonglon - khachhangthanthiet - voucher)
        return tong
    def __set_new_score(self, bill_total) -> None:
        new_point = int(bill_total * 0.1)
        self.account.add_score(new_point)
    
    # in bill (luu vao file txt: bill_<order_id>.txt)
    def printBill(self):
        total_cost = self.calculate_total()

        with open(f"bill_{self.order_id}.txt", "w") as file:
            file.write(f"Bill ID: {self.order_id}\n")
            file.write(f"Voucher: {self.voucher}\n")
            file.write(f"Loyalty Discount: {self.account.getLoyaltyPromo() * 100}%\n")
            file.write(f"Bulk Order Discount: {0.1 if self.item_list.get_sum_qty() >= 10 else 0  * 100}%\n")
            file.write(f"Total (after discounts): ${total_cost:.2f}\n")
            file.write(f"Score (after order): {self.account.score}\n")
            file.write("\nItems:\n")
            file.write(self.item_list)